DRKG

Adapted from: https://github.com/gnn4dr/DRKG/blob/master/drkg_with_dgl/loading_drkg_in_dgl.ipynb

In [6]:
import pandas as pd
import numpy as np
import os 
import torch_geometric.transforms as T
import torch
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as pyg
import torch_geometric
from torch_geometric.nn import SAGEConv, to_hetero
from   torch.utils.data      import Dataset, DataLoader
from   torch_geometric.data  import Data
from   torch_geometric.utils import negative_sampling


from torch_geometric.nn           import SAGEConv, to_hetero
from SIMP_LLM.dataloader_mappings import load_graph,load_csv_as_list


from SIMP_LLM.models              import  GATStack,GNNStack,LinkPredictorMLP,CosineSimilarityModel,save_torch_model
from SIMP_LLM.train_utils         import  forward_pass, train,evaluate,   filter_edge_index_by_value,split_edge_index,get_negative_edges

import matplotlib.pyplot as plt #needed to visualize loss curves
import numpy as np 

from sklearn         import metrics
from sklearn.metrics import ConfusionMatrixDisplay


import copy

In [36]:
import pickle

# Specify the path to your pickle file
pickle_file_path = 'data/rare_disease_data.pkl'

# Open the pickle file in read mode
with open(pickle_file_path, 'rb') as file:
    # Load the contents of the pickle file
    data = pickle.load(file)

    
drug_ids    = [data['drug_id'] for data in data]
disease_ids = [data['disease_id'] for data in data]

# Create edge index tensor
edge_index_pre = torch.tensor([drug_ids, disease_ids])

# Print the edge index tensor
print(edge_index_pre)


tensor([[ 2574, 11387,  4595, 14540, 14540, 14540,  1834,  6994,  6994,  2121,
          1339,  7556,  1072, 11813, 11813, 11813,   145,  1148, 10332,  6374,
          9680,  9680,  4390,  7837,  4692, 10159, 10758,  7162,  2366, 10412,
         11908, 13763],
        [  483,  2205,  2449,  3460,  3522,  3460,  2449,   357,   357,   483,
          2449,  1005,  2449,  3460,  3522,  3460,    76,    76,   856,    76,
           375,   375,   973,  2449,  2449,  2449,  2449,  2449,  2449,   483,
           483,  2449]])


In [37]:
  data 

[{'ID': 'Alpelisib->breast cancer',
  'drug_name': 'Alpelisib',
  'disease_name': 'breast cancer',
  'drug_id': 2574,
  'disease_id': 483},
 {'ID': 'Ambenonium->Myasthenia Gravis',
  'drug_name': 'Ambenonium',
  'disease_name': 'Myasthenia Gravis',
  'drug_id': 11387,
  'disease_id': 2205},
 {'ID': 'Apalutamide->prostate cancer',
  'drug_name': 'Apalutamide',
  'disease_name': 'prostate cancer',
  'drug_id': 4595,
  'disease_id': 2449},
 {'ID': 'Artesunate->Malaria',
  'drug_name': 'Artesunate',
  'disease_name': 'Malaria',
  'drug_id': 14540,
  'disease_id': 3460},
 {'ID': 'Artesunate->malaria',
  'drug_name': 'Artesunate',
  'disease_name': 'malaria',
  'drug_id': 14540,
  'disease_id': 3522},
 {'ID': 'Artesunate->Malaria',
  'drug_name': 'Artesunate',
  'disease_name': 'Malaria',
  'drug_id': 14540,
  'disease_id': 3460},
 {'ID': 'Darolutamide->prostate cancer',
  'drug_name': 'Darolutamide',
  'disease_name': 'prostate cancer',
  'drug_id': 1834,
  'disease_id': 2449},
 {'ID': 'Del

## Load Graph

In [8]:
#triplets =[('Compound', 'Compound_treats_the_disease', 'Disease')] # Uncomment this line 
triplets     = load_csv_as_list('triplets.csv')                     # triplets of form (tail, relationship,head)
data         = load_graph(triplets )                                # Load Graph based on triplets (You can selet the triplets 
dictionaries = torch.load("data2/mapping_dict")
#print(data)
#print(data["Compound", "Compound_treats_the_disease", "Disease"].edge_index)

data2


## GRAPH SAGE

#### 1 Masking Sanity Check

In [9]:
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2, 2, 3,4,5,5,5,5,5], [1, 2, 3, 4, 5, 6, 7, 8,9,10,11,12,13,14]])
print(f"Oringal Edge index:\n{edge_index }\n: {edge_index.shape}\n")
edge_index_1 , edge_index_2 = split_edge_index(edge_index ,percentage = 0.5,verbose=True)

Oringal Edge index:
tensor([[ 0,  0,  0,  1,  1,  2,  2,  3,  4,  5,  5,  5,  5,  5],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]])
: torch.Size([2, 14])

##############
Edge Index 1: torch.Size([2, 5])

Edge Index 2: torch.Size([2, 9])
##############


### 2 Mask Real Data 

###  3 Compile Model 

In [10]:
epochs        = 500
hidden_dim    = 524      
dropout       = 0.7
num_layers    = 3
learning_rate = 1e-4
node_emb_dim  = 768
device        = "cpu"
head          = "COSINE"    # one of   "MLP", "COSINE"
backbone      = "GraphSage" # one of  "GraphSage","GAT"



#GATStack,GNNStack


if backbone == "GraphSage":
    HomoGNN         = GNNStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, return_embedding=True).to(device) # the graph neural network that takes all the node embeddings as inputs to message pass and agregate
elif backbone == "GAT":
    HomoGNN         = GATStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, return_embedding=True).to(device) 
    
embed_model     = to_hetero(HomoGNN   , data.metadata(), aggr='sum')
if head == "MLP":
    link_predictor  = LinkPredictorMLP(hidden_dim, hidden_dim, 1, num_layers , dropout).to(device) # the MLP that takes embeddings of a pair of nodes and predicts the existence of an edge between them
if head == "COSINE":
    link_predictor = CosineSimilarityModel(input_dim=524 ).to(device)

#optimizer      = torch.optim.AdamW(list(model.parameters()) + list(link_predictor.parameters() ), lr=learning_rate, weight_decay=1e-4)
if head ==  "MLP":
    optimizer       = torch.optim.Adam(list(embed_model.parameters()) + list(link_predictor.parameters() ), lr=learning_rate)
elif head == "COSINE":
    optimizer       = torch.optim.Adam(list(embed_model.parameters()) , lr=learning_rate)
    

#print(HeteroGNN )
print(link_predictor)
print(f"Models Loaded to {device}")
data.to(device)
embed_model.to(device)
print()

/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Side Effect' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Biological Process' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Cellular Component' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_

CosineSimilarityModel(
  (cos): CosineSimilarity()
)
Models Loaded to cpu



In [11]:
#### Load Best Models ####
best_graphsage_model_path      = os.path.join("Saved_models","GraphSageCosine","GraphSage_epoch_58.pt")
best_link_predictor_model_path = os.path.join("Saved_models","GraphSageCosine","link_predictor_epoch_58.pt")
print(f"Loading best models:  {best_graphsage_model_path }  {best_link_predictor_model_path}")
                                              
                                              
checkpoint = torch.load(best_graphsage_model_path  )
embed_model.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load(best_link_predictor_model_path)
link_predictor.load_state_dict(checkpoint['model_state_dict'])

del checkpoint


Loading best models:  Saved_models/GraphSageCosine/GraphSage_epoch_58.pt  Saved_models/GraphSageCosine/link_predictor_epoch_58.pt


In [13]:
 node_emb   = embed_model(data.x_dict, data.edge_index_dict)  # Embed Bert Embeddigns with graphsage (N, d) 

In [ ]:
edge_index_pre

In [19]:

head                = node_emb['Compound'][drug_ids]
tail                = node_emb['Disease'][disease_ids]
pred                = link_predictor( head  ,    tail   )

In [33]:
(pred > 0.3254*2)*1

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1])

In [30]:
((pred > 0.3254)*1).sum()/(len(pred))


tensor(0.3750)

In [34]:
pred

tensor([0.2965, 0.2909, 0.2913, 0.2906, 0.2949, 0.2906, 0.2952, 0.2910, 0.2910,
        0.3023, 0.2957, 0.6979, 0.7050, 0.2941, 0.2956, 0.2941, 0.2882, 0.2923,
        0.7010, 0.2937, 0.3173, 0.3173, 0.2949, 0.7020, 0.7069, 0.7059, 0.5739,
        0.7034, 0.7028, 0.6972, 0.6972, 0.7050], grad_fn=<SigmoidBackward0>)